In [ ]:
%matplotlib inline
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import sys
import datetime
    
    
def plot_fig(filename, label, band):
    ds = gdal.Open(filename, gdal.GA_ReadOnly)
    data = ds.GetRasterBand(band).ReadAsArray()
    ds = None
    print(np.min(data), np.max(data))
    fig,ax = plt.subplots(figsize=(11,10))
    im = ax.imshow(data, cmap='jet')
    fig.colorbar(im)
    ax.set_title(label)
    fig.show()

## set path

In [ ]:
merged_dir = '/media/ly/文件/stacking/xsc/merged'
gacos_dir = '/media/ly/文件/stacking/xsc/gacos'

## get box

In [ ]:
os.chdir(os.path.join(merged_dir, 'geom_master'))
box = ''

ds = gdal.Open('lat.rdr', gdal.GA_ReadOnly)
data = ds.GetRasterBand(1).ReadAsArray()
box += (str(np.min(data)) + ' ')
box += (str(np.max(data)) + ' ')
data = None

ds = gdal.Open('lon.rdr', gdal.GA_ReadOnly)
data = ds.GetRasterBand(1).ReadAsArray()
box += (str(np.min(data)) + ' ')
box += str(np.max(data))
data = None

print(box)

## geocode los.rdr

In [ ]:
os.chdir(os.path.join(merged_dir, 'geom_master'))

cmd_str = f"geocodeGdal.py -l lat.rdr -L lon.rdr -f los.rdr -b '{box}'"
os.system(cmd_str)

## geocode unw

In [ ]:
unws = glob.glob(os.path.join(merged_dir, 'interferograms/*/filt_fine.unw'))
lat = os.path.join(merged_dir, 'geom_master', 'lat.rdr')
lon = os.path.join(merged_dir, 'geom_master', 'lon.rdr')

for unw in unws:
    dir_name = os.path.dirname(unw)
    os.chdir(dir_name)
    sys.stdout.write(f"\rprocess {unws.index(unw) + 1}/{len(unws)}")
    sys.stdout.flush()
    cmd_str = f"geocodeGdal.py -l ./../../geom_master/lat.rdr -L ./../../geom_master/lon.rdr -f filt_fine.unw -b '{box}'"
    os.system(cmd_str)

In [ ]:
geo_unws = glob.glob(os.path.join(merged_dir, 'interferograms/*/geo_filt_fine.unw'))
plot_fig(geo_unws[0], 'unwraped', 2)

## gacos rsc2hdr

In [ ]:
def loadrsc(infile):
    '''A function to load the content of .rsc file and pass it back as a dictionary'''
    import numpy as np
    with open(infile + '.rsc') as f:
        text = f.read()
    lines = [e.split() for e in text.split("\n") if e != ""]
    headers = dict(lines)
    # add the filename such it can be called when making envi header
    headers['FILENAME'] = infile
    # take the abs of the y-spacing as upper left corner is to be specified
    headers['Y_STEP'] = str(np.abs(float(headers['Y_STEP'])))
    return headers

def writehdr(filename,headers):
    '''A function that writes a .hdr file from a template and a dictionarydescribing the fields'''
    print('Writing output HDR file...')
    print(headers)
    enviHDRFile = open(filename + '.hdr', 'w')
    enviHDR = '''ENVI
description = {{GACOS: {FILENAME} }}
samples = {WIDTH}
lines = {FILE_LENGTH}
bands = 1
header offset = 0
file type = ENVI Standard
data type = 4
interleave = bsq
sensor type = Unknown
byte order = 0
map info = {{Geographic Lat/Lon, 1, 1, {X_FIRST}, {Y_FIRST}, {X_STEP}, {Y_STEP}, WGS-84, units=Degrees}}
coordinate system string = {{GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.017453292519943295]]}}'''.format(**headers)

    enviHDRFile.write(enviHDR)
    enviHDRFile.close()
    print('Output HDR file =', filename)

def GACOS_rsc2hdr(inputfile):
    '''
    Wrapper code which calls .rsc reader and .hdr writer functionality 
    '''

    print('Generating hdr file for: ' + inputfile + '...')
    # make sure the user does not give a header file as input
    filename, file_extension = os.path.splitext(inputfile)
    if file_extension == '.hdr' or file_extension == '.rsc':
        raise Exception("Give path to the ENVI file not the .hdr or .rsc file")
    headers = loadrsc(inputfile)
    writehdr(inputfile,headers)
    print('hdr for ' + inputfile + ' generated')
    print(" ")

In [ ]:
ztds = glob.glob(os.path.join(gacos_dir, '*.ztd'))
for ztd in ztds:
    GACOS_rsc2hdr(ztd)

## file transform

In [ ]:
def file_transform(unwfile,apsfile,apsfile_out):
    '''
    convert the aps file into the same geo frame as the unw file
    Unwfile is an envi file and has a corresponding vrt file
    aps file is gdal compatible
    '''
    import isceobj
    from osgeo import gdal, gdalconst
    #from gdal2isce_xml import gdalisce2XML

    # convert all to absolute paths
    apsfile = os.path.abspath(apsfile)
    apsfile_out = os.path.abspath(apsfile_out)

    # Source
    src = gdal.Open(apsfile, gdalconst.GA_ReadOnly)
    src_proj = src.GetProjection()
    src_geotrans = src.GetGeoTransform()
    print("Working on " + apsfile )
    # We want a section of source that matches this:
    match_ds = gdal.Open(unwfile + '.vrt', gdalconst.GA_ReadOnly)
    match_proj = match_ds.GetProjection()
    match_geotrans = match_ds.GetGeoTransform()
    print("Getting target reference information")
    wide = match_ds.RasterXSize
    high = match_ds.RasterYSize
    
    # Output / destination
    dst = gdal.GetDriverByName('envi').Create(apsfile_out, wide, high, 1, gdalconst.GDT_Float32)
    dst.SetGeoTransform( match_geotrans )
    dst.SetProjection( match_proj)

    # Do the work
    gdal.ReprojectImage(src, dst, src_proj, match_proj, gdalconst.GRA_Bilinear)
    print("Done")
    print("")


    # closing the images
    dst = None
    src = None

    ## run gdal 2 isce on this file so we can use ISCE functionality latter on
    #gdalisce2XML(apsfile_out)
    

In [ ]:
geo_unws = glob.glob(os.path.join(merged_dir, 'interferograms/*/geo_filt_fine.unw'))
# plot_fig(geo_unws[0], 'unwraped', 2)


ztds = glob.glob(os.path.join(gacos_dir, '*.ztd'))
for ztd in ztds:
    ztd_geo = ztd + '.geo'
    file_transform(geo_unws[0], ztd, ztd_geo)

In [ ]:
ztd_geos = glob.glob(os.path.join(gacos_dir, '*.ztd.geo'))
plot_fig(ztd_geos[0], 'ztd.geo', 1)

## zenith2slant

In [ ]:
 def zenith2slant(losfile,aps_zenith,aps_slant):
    
    # convert all to absolute paths
    aps_zenith = os.path.abspath(aps_zenith)
    aps_slant = os.path.abspath(aps_slant)
    losfile = os.path.abspath(losfile)
    
    # loading the zenith APS file
    ds = gdal.Open(aps_zenith, gdal.GA_ReadOnly)
    zenith = ds.GetRasterBand(1).ReadAsArray()
    proj = ds.GetProjection()
    geotrans = ds.GetGeoTransform()
    ds = None
    
    # loading the incidence angle file
    ds = gdal.Open(losfile, gdal.GA_ReadOnly)
    inc = ds.GetRasterBand(1).ReadAsArray()
    ds = None
    # convert the inc from deg to rad
    inc = inc*np.pi/180
    
    # scaling factor to convert pseudo-range [m] increase to phase delay [rad]
    scaling = -4*np.pi/5.6*100
    
    # projecting the zenith into the slant
    slant = scaling*zenith/np.cos(inc)
    
    # making sure the no-date is propagated
    slant[zenith==0]=0
    slant[inc==0]=0
        
    
    # writing out the file   
    drv = gdal.GetDriverByName('envi').Create(aps_slant, slant.shape[1], slant.shape[0], 1,gdal.GDT_Float32)
    drv.SetGeoTransform(geotrans)
    drv.SetProjection(proj)
    drv.GetRasterBand(1).WriteArray(slant)
    drv = None
    

In [ ]:
ztd_geos = glob.glob(os.path.join(gacos_dir, '*.ztd.geo'))
geo_los = os.path.join(merged_dir, 'geom_master/geo_los.rdr')
for ztd_geo in ztd_geos:
    aps_geo = ztd_geo.replace('ztd', 'aps')
    sys.stdout.write(f"\rprocess {ztd_geos.index(ztd_geo) + 1}/{len(ztd_geos)}")
    sys.stdout.flush()
    zenith2slant(geo_los, ztd_geo, aps_geo)

In [ ]:
aps_geos = glob.glob(os.path.join(gacos_dir, '*.aps.geo'))
plot_fig(aps_geos[0], 'aps.geo', 1)

## differential delay

In [ ]:
def differential_delay(master_aps,slave_aps,outname):
    
    # convert all to absolute paths
    master_aps = os.path.abspath(master_aps)
    slave_aps = os.path.abspath(slave_aps)
    outname = os.path.abspath(outname)
    
    # loading the master APS file
    ds = gdal.Open(master_aps, gdal.GA_ReadOnly)
    master = ds.GetRasterBand(1).ReadAsArray()
    proj = ds.GetProjection()
    geotrans = ds.GetGeoTransform()
    ds = None
    
    # loading the slave APS file
    ds = gdal.Open(slave_aps, gdal.GA_ReadOnly)
    slave = ds.GetRasterBand(1).ReadAsArray()
    ds = None
    
    
    # computing the differential APS
#     diffAPS = master-slave
    diffAPS = slave-master
    
    # writing out the file 
    drv = gdal.GetDriverByName('envi').Create(outname, diffAPS.shape[1], diffAPS.shape[0], 1,gdal.GDT_Float32)
    drv.SetGeoTransform(geotrans)
    drv.SetProjection(proj)
    drv.GetRasterBand(1).WriteArray(diffAPS)
    drv = None

In [ ]:
ifgs = glob.glob(os.path.join(merged_dir, 'interferograms/*'))
for ifg in ifgs:
    master = ifg[-17:-9]
    slave = ifg[-8::]
    master_aps = os.path.join(gacos_dir, master + '.aps.geo')
    slave_aps = os.path.join(gacos_dir, slave + '.aps.geo')
#     diff_aps = os.path.join(ifg, master + '_' + slave + '.aps.geo')
    diff_aps = os.path.join(ifg, 'diff.aps.geo')
    sys.stdout.write(f"\rprocess {ifgs.index(ifg) + 1}/{len(ifgs)}")
    sys.stdout.flush()
    differential_delay(master_aps, slave_aps, diff_aps)

In [ ]:
diff_aps = glob.glob(os.path.join(merged_dir, 'interferograms/*/*.aps.geo'))
unws = glob.glob(os.path.join(merged_dir, 'interferograms/*/geo_filt_fine.unw'))
plot_fig(diff_aps[0], 'diff.aps.geo', 1)
plot_fig(unws[0], 'unw', 2)

## ifg correction

In [ ]:
def ifg_correction(unw,aps,outname):
    
    # convert all to absolute paths
    unw = os.path.abspath(unw)
    aps = os.path.abspath(aps)
    outname = os.path.abspath(outname)
    
    # loading the UNW file
    ds = gdal.Open(unw, gdal.GA_ReadOnly)
    unwdata_phase = ds.GetRasterBand(2).ReadAsArray()
    unwdata_amplitude = ds.GetRasterBand(1).ReadAsArray()
    proj = ds.GetProjection()
    geotrans = ds.GetGeoTransform()
    ds = None
    
    # loading the APS file
    ds = gdal.Open(aps, gdal.GA_ReadOnly)
    apsdata = ds.GetRasterBand(1).ReadAsArray()
    ds = None
    
    # Correcting the IFG
    unwdata_phase = unwdata_phase - apsdata
    # making sure the no-date is propagated
    unwdata_phase[unwdata_phase==0]=0
    unwdata_phase[apsdata==0]=0 
    
    
    # writing out the file 
    drv = gdal.GetDriverByName('envi').Create(outname, unwdata_phase.shape[1], unwdata_phase.shape[0], 2,gdal.GDT_Float32)
    drv.SetGeoTransform(geotrans)
    drv.SetProjection(proj)
    drv.GetRasterBand(1).WriteArray(unwdata_amplitude)
    drv.GetRasterBand(2).WriteArray(unwdata_phase)
    drv = None
    

In [ ]:
ifgs = glob.glob(os.path.join(merged_dir, 'interferograms/*'))
for ifg in ifgs:
    name = ifg[-17::]
    unw = os.path.join(ifg, 'geo_filt_fine.unw')
    aps = os.path.join(ifg, name + '.aps.geo')
    corrected_unw = os.path.join(ifg, 'geo_filt_fine.aps.unw')
    sys.stdout.write(f"\rprocess {ifgs.index(ifg) + 1}/{len(ifgs)}")
    sys.stdout.flush()
    ifg_correction(unw, aps, corrected_unw)

In [ ]:
def plot_fig2(unw, aps_unw):
    ds = gdal.Open(unw, gdal.GA_ReadOnly)
    unw = ds.GetRasterBand(2).ReadAsArray()
    ds = None

    ds = gdal.Open(aps_unw, gdal.GA_ReadOnly)
    aps_unw = ds.GetRasterBand(2).ReadAsArray()
    ds = None
    
    fig,ax = plt.subplots(figsize=(11,10), nrows=1, ncols=2)
    im = ax[0].imshow(unw, cmap='jet', vmin=-100, vmax=100)
    ax[0].set_title('unw')
    
    im = ax[1].imshow(aps_unw, cmap='jet', vmin=-100, vmax=100)
    ax[1].set_title('aps_unw')
    fig.colorbar(im)
    fig.show()


unws = glob.glob(os.path.join(merged_dir, 'interferograms/*/geo_filt_fine.unw'))
aps_unws = glob.glob(os.path.join(merged_dir, 'interferograms/*/geo_filt_fine.aps.unw'))

# plot_fig(aps_unws[0], 'aps.unw', 2)
# plot_fig(unws[0], 'unw', 2)
plot_fig2(unws[2], aps_unws[2])

## stacking phase

In [ ]:
def differential_delay(master_aps,slave_aps,outname):
    
    # convert all to absolute paths
    master_aps = os.path.abspath(master_aps)
    slave_aps = os.path.abspath(slave_aps)
    outname = os.path.abspath(outname)
    
    # loading the master APS file
    ds = gdal.Open(master_aps, gdal.GA_ReadOnly)
    master = ds.GetRasterBand(1).ReadAsArray()
    proj = ds.GetProjection()
    geotrans = ds.GetGeoTransform()
    ds = None
    
    # loading the slave APS file
    ds = gdal.Open(slave_aps, gdal.GA_ReadOnly)
    slave = ds.GetRasterBand(1).ReadAsArray()
    ds = None
    
    
    # computing the differential APS
    diffAPS = master-slave
    
    # writing out the file 
    drv = gdal.GetDriverByName('envi').Create(outname, diffAPS.shape[1], diffAPS.shape[0], 1,gdal.GDT_Float32)
    drv.SetGeoTransform(geotrans)
    drv.SetProjection(proj)
    drv.GetRasterBand(1).WriteArray(diffAPS)
    drv = None
    
def gen_ifg_pairs(slc_date, num_connections):
    slc_date = sorted(slc_date)
    ifg_pairs = []
    length = len(slc_date)
    for i in range(length):
        if i < length - num_connections:
            for j in range(num_connections):
                ifg_pairs.append(f"{slc_date[i]}_{slc_date[i + j + 1]}")
        else:
            for k in range(length - i - 1):
                ifg_pairs.append(f"{slc_date[i]}_{slc_date[i + k + 1]}")
    return ifg_pairs

def get_slc_date(ifg_dir):
    slc_date = []
    ifgs = glob.glob(os.path.join(ifg_dir, '*'))
    for ifg in ifgs:
        slc_date.append(ifg[-17:-9])
        slc_date.append(ifg[-8::])
    return list(set(slc_date))

def get_days(ifg_pair):
    master = ifg_pair.split('_')[0]
    slave = ifg_pair.split('_')[1]
    m_date = datetime.datetime.strptime(master, "%Y%m%d")
    s_date = datetime.datetime.strptime(slave, "%Y%m%d")
    delta = (s_date - m_date).days
    return delta
    

def stacking(ifg_dir, stack_file, out_file):
    avg_phase = 0
    slc_date = get_slc_date(ifg_dir)
    ifg_pairs = sorted(os.listdir(ifg_dir))
#     ifg_pairs = gen_ifg_pairs(slc_date, 1)
    phase2range = -5.6 / (4.0 * np.pi)
    for ifg_pair in ifg_pairs:
        sys.stdout.write('\rprocess {}/{}'.format(ifg_pairs.index(ifg_pair) + 1, len(ifg_pairs)))
        sys.stdout.flush()
        aps_unw = os.path.join(ifg_dir, ifg_pair, stack_file)
        ds = gdal.Open(aps_unw, gdal.GA_ReadOnly)
        data = ds.GetRasterBand(2).ReadAsArray()
        proj = ds.GetProjection()
        geotrans = ds.GetGeoTransform()
        ds = None
        tbase = get_days(ifg_pair) / 365
        data *= phase2range / tbase
        avg_phase += data
    avg_phase /= len(ifg_pairs)
    # writing out the file 
    drv = gdal.GetDriverByName('envi').Create(out_file, avg_phase.shape[1], avg_phase.shape[0], 1,gdal.GDT_Float32)
    drv.SetGeoTransform(geotrans)
    drv.SetProjection(proj)
    drv.GetRasterBand(1).WriteArray(avg_phase)
    drv = None
    
    

    
    
def stacking2(ifg_dir, stack_file, out_file):
    avg_phase = 0
    slc_date = get_slc_date(ifg_dir)
    ifg_pairs = sorted(os.listdir(ifg_dir))
    sum_all = sum([get_days(ifg_pair) / 365 * get_days(ifg_pair) / 365 for ifg_pair in ifg_pairs])
    phase2range = -5.6 / (4.0 * np.pi)
    for ifg_pair in ifg_pairs:
        sys.stdout.write('\rprocess {}/{}'.format(ifg_pairs.index(ifg_pair) + 1, len(ifg_pairs)))
        sys.stdout.flush()
        aps_unw = os.path.join(ifg_dir, ifg_pair, stack_file)
        ds = gdal.Open(aps_unw, gdal.GA_ReadOnly)
        data = ds.GetRasterBand(2).ReadAsArray()
        proj = ds.GetProjection()
        geotrans = ds.GetGeoTransform()
        ds = None
        tbase = get_days(ifg_pair) / 365
        data *= (phase2range * tbase / sum_all)
        avg_phase += data
    # writing out the file 
    drv = gdal.GetDriverByName('envi').Create(out_file, avg_phase.shape[1], avg_phase.shape[0], 1,gdal.GDT_Float32)
    drv.SetGeoTransform(geotrans)
    drv.SetProjection(proj)
    drv.GetRasterBand(1).WriteArray(avg_phase)
    drv = None

In [ ]:
ifg_dir = os.path.join(merged_dir, 'interferograms')
out_file = os.path.join(merged_dir, 'avg_aps')
stack_file = 'diff.aps.geo'
stacking(ifg_dir, stack_file, out_file)

In [ ]:
os.chdir(merged_dir)
plot_fig(out_file, 'avg_aps', 1)
plot_fig('avg_phase', 'avg_phase', 1)

## save kml

In [ ]:
os.chdir(merged_dir)
!saveKml.py -f avg_phase_aps -m -23 -M 5 -d 600 -c jet -u cm/yr -s 1 -b 20200808 -e 20200809 -n 1